In [1]:
from bs4 import BeautifulSoup
import requests
import pprint
import pandas as pd
import json

In [2]:
url = "https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=1&sort=priceLowest&bodyStyle=Convertible"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

In [3]:
soup.find("div", class_="search-page-grid-card--vehicle-details ng-star-inserted").find('a').get('href')

'/cars/vehicle-details/2011/Volvo/C70/Base/YV1672MC2BJ117253'

In [4]:
def get_car_name(element):
    name=None
    for i in element.findChildren():
        if i.find("div", class_="grv-text--ellipsis grv-text--normal-semibold"):
            name = i.find("div", class_="grv-text--ellipsis grv-text--normal-semibold").get_text()
    return name

In [5]:
def get_car_model(element):
    model=None
    for i in element.findChildren():
        if i.find("div", class_="grv-text--ellipsis grv-color--digital-gray-90"):
            model = i.find("div", class_="grv-text--ellipsis grv-color--digital-gray-90").get_text()
    return model

In [6]:
def get_car_price(element): 
    price=None
    for i in element.findChildren():
        if i.find("div", class_="grv-text--tiny-semibold"):
            price = i.find("div", class_="grv-text--tiny-semibold").find('a').get_text()
    return price

In [21]:
#def get_car_price(element):
#    price=None
#    for i in element.findChildren():
#        if i.find("div", class_="vehicle-value__price ng-star-inserted"):
#            price = i.find("div", class_="vehicle-value__price ng-star-inserted").get_text()
#    return price

In [7]:
def get_car_dealer(element):
    dealer=None
    for i in element.findChildren():
        if i.find("div", class_="search-page-grid-card--vehicle-price-location__dealer"):
            dealer = i.find("div", class_="search-page-grid-card--vehicle-price-location__dealer").get_text()
    return dealer

In [8]:
def get_car_milage(element):
    milage=None
    for i in element.findChildren():
        if i.find("span", class_="ng-star-inserted"):
            for j in i.find_all("span", class_="ng-star-inserted"):
                milage = j.get_text()
    return milage

In [9]:
def get_other_feature_list(element):
    features=[]
    for i in element.find_all("div", class_="other-features grv-col--sm-4 grv-col--md-4 grv-col--lg-3 ng-star-inserted"):
        features.append(i.get_text())
    return features

def get_feature_dict(page_element):
    features={}
    for i in page_element.find_all("div",class_="detail-tile grv-col--sm-2 grv-col--md-4 grv-col--lg-3"):
        features[i.find("div",class_="detail-key").get_text()] = i.find("div",class_="detail-val").get_text()
    
    features.update({"Other Features" : list(get_other_feature_list(page_element))})
    return features

In [10]:
def get_vehical_details(card):
    features={}
    url = "https://www.capitalone.com/"+card.find('a').get('href')
    req = requests.get(url)
    car_html = BeautifulSoup(req.content, 'html.parser')
    features = get_feature_dict(car_html)
    return features

In [11]:
def get_page_data(html):
    name,model,price,dealer,milage,temp_list,feature_dict,df_dict = [],[],[],[],[],[],{},{}
    feature_list=['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
    
    for i in html.find_all("div", class_="search-page-grid-card--vehicle-details ng-star-inserted"):
        temp=[]
        name.append(get_car_name(i))
        model.append(get_car_model(i))
        price.append(get_car_price(i))
        dealer.append(get_car_dealer(i))
        milage.append(get_car_milage(i))
        mydict=get_vehical_details(i)
        temp= list(mydict.values())
        temp_list.append(temp)
        
    df_dict["Name"]=name
    df_dict["Model"]=model
    df_dict["Price"]=price
    df_dict["Dealer"]=dealer
    df_dict["Used"]=milage
    print(feature_list)
    
    for i in feature_list:
        df_dict[i]=[]

    for idx,i in enumerate(temp_list):
        for jdx,j in enumerate(feature_list):
            df_dict[j].append(i[jdx])
    
    df = pd.DataFrame(df_dict)
    return df

def get_site_data(page_no):
    df=pd.DataFrame()
    url = "https://www.capitalone.com/cars/find/New+York-NY?radius=50&page="+page_no+"&sort=priceLowest&bodyStyle=Convertible"
    req = requests.get(url)
    html = BeautifulSoup(req.content, 'html.parser')
    df = get_page_data(html)
    print(url)
    return df

In [12]:
df = pd.DataFrame()
for i in range(1,6):
    df = df.append(get_site_data(str(i)),ignore_index=True)

['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=1&sort=priceLowest&bodyStyle=Convertible


C:\Users\Aishwary\AppData\Local\Temp\ipykernel_2136\4046322587.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_site_data(str(i)),ignore_index=True)


['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=2&sort=priceLowest&bodyStyle=Convertible
['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=3&sort=priceLowest&bodyStyle=Convertible
['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=4&sort=priceLowest&bodyStyle=Convertible
['Condition', 'Mileage', 'MPG', 'Body Style', 'Exterior Color', 'Interior Color', 'Transmission', 'Drive Train', 'Fuel Type', 'Engine', 'Other Features']
https://www.capitalone.com/cars/find/New+York-NY?radius=50&page=5&sor

In [14]:
df

,Name,Model,Price,Dealer,Used,Condition,Mileage,MPG,Body Style,Exterior Color,Interior Color,Transmission,Drive Train,Fuel Type,Engine,Other Features
0,2011 Volvo,C70 Base,None,None,"123,667 mi",USED,"123,667",19 City / 28 Hwy,Convertible,Blue,Brown,Automatic,FWD,Gasoline,5 Cyl,"[ 17 Inch Wheels , 3-Point Seat Belts , 4-Wh..."
1,2012 Fiat,500 Lounge,None,None,"84,340 mi",USED,"84,340",27 City / 32 Hwy,Convertible,White,––,Automatic,FWD,Gasoline,4 Cyl,"[ 15 Inch Wheels , 4-Wheel Disc Brakes , AM/..."
2,2013 Mini,Cooper Convertible Base,None,None,"114,249 mi",USED,"114,249",27 City / 35 Hwy,Convertible,Red,White,Manual,FWD,Gasoline,4 Cyl,"[ Adjustable Headrests , Adjustable Lumbar Su..."
3,2013 Fiat,500 Abarth,None,None,"84,294 mi",USED,"84,294",28 City / 34 Hwy,Convertible,Black,Black,Manual,FWD,Gasoline,4 Cyl,"[ 16 Inch Wheels , 4-Wheel Disc Brakes , AM/..."
4,2012 Audi,A5 2.0T Premium Plus,None,None,"95,800 mi",USED,"95,800",21 City / 29 Hwy,Convertible,Black,Black,Automatic,AWD,Gasoline,4 Cyl,"[ 18 Inch Wheels , 4-Wheel Disc Brakes , AM/..."
5,2014 Fiat,500C GQ Edition,None,None,"46,776 mi",USED,"46,776",––,Convertible,White,Black,Manual,FWD,Gasoline,4 Cyl,"[ 2-Stage Unlocking , 3-Point Seat Belts , 4..."
6,2014 Fiat,500C GQ Edition,None,None,"44,929 mi",USED,"44,929",––,Convertible,Silver,Black,Manual,FWD,Gasoline,4 Cyl,"[ 4-Wheel Disc Brakes , Airbags , Anti-Lock ..."
7,2012 BMW,1 Series 128i,None,None,"51,083 mi",USED,"51,083",18 City / 28 Hwy,Convertible,Blue,White,Automatic,RWD,Gasoline,6 Cyl,"[ 4-Wheel Disc Brakes , 4-Wheel Independent S..."
8,2011 Audi,A5 2.0T Premium,None,None,"62,994 mi",USED,"62,994",21 City / 29 Hwy,Convertible,Black,Tan,Automatic,AWD,Gasoline,4 Cyl,"[ Adjustable Steering Wheel , Air Conditionin..."
9,2014 Volkswagen,Beetle 2.5L Technology,None,None,"91,633 mi",USED,"91,633",21 City / 27 Hwy,Convertible,White,Black,Automatic,FWD,Gasoline,5 Cyl,"[ 17 Inch Wheels , 18 Inch Wheels , 4-Wheel ..."


In [29]:
def get_other_feature_list(element):
    features=[]
    for i in element.find_all("div", class_="other-features grv-col--sm-4 grv-col--md-4 grv-col--lg-3 ng-star-inserted"):
        features.append(i.get_text())
    
    return features

In [30]:
df.to_csv('data_convertible.csv',index=False)